In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [ ]:
from pythermalcomfort.models import pmv_ppd
from pythermalcomfort.utilities import v_relative, clo_dynamic
from pythermalcomfort.utilities import met_typical_tasks
from pythermalcomfort.utilities import clo_individual_garments
tdb = 27  # dry bulb air temperature, [$^{\circ}$C]
tr = 25  # mean radiant temperature, [$^{\circ}$C]
v = 0.3  # average air speed, [m/s]
rh = 50  # relative humidity, [%]
activity = "Typing"  # participant's activity description
garments = ["Sweatpants", "T-shirt", "Shoes or sandals"]

met = met_typical_tasks[activity]  # activity met, [met]
icl = sum(
    [clo_individual_garments[item] for item in garments]
)  # calculate total clothing insulation

# calculate the relative air velocity
vr = v_relative(v=v, met=met)
# calculate the dynamic clothing insulation
clo = clo_dynamic(clo=icl, met=met)

# calculate PMV in accordance with the ASHRAE 55 2020
results = pmv_ppd(tdb=tdb, tr=tr, vr=vr, rh=rh, met=met, clo=clo, standard="ASHRAE")

In [ ]:
T = 5 # demand response time (h)
N = 5 # the building consisting of N floors
Population = np.zeros((N,T))
for i in range(N):
    for j in range(T):
        Population[i,j] = 70*(i+1) - 5*j # population distribution in the building
to = np.array([20+i for i in range(T)]) # outdoor air temperature

In [ ]:
from asyncio import constants
from pyomo.environ import * 
modelA = ConcreteModel()
DRdemand = -50
# declare dicision variables
modelA.T = RangeSet(0,T-1)
modelA.N = RangeSet(0,N-1)
modelA.deltaTset=Var(modelA.N, modelA.T, domain = Reals, initialize=-1, bounds=(-3,0))
# modelA.deltaTset.pprint()

# declare constraints
modelA.con1 = Constraint(expr= sum(modelA.deltaTset[i,j] for i in range(N) for j in range(T) ) <= DRdemand)

#declare objective function
def ComfortLoss(model):
    """
    calculate the comfort loss matrix
    """
    loss = sum( Population[i,j] * model.deltaTset[i,j] for i in range(N) for j in range(T) )
    return abs(loss)
modelA.obj = Objective(rule = ComfortLoss, sense=minimize)

# modelA.pprint()
SolverFactory('ipopt').solve(modelA)
modelA.display()
